##The purpose of this script is to get a tree file together with all of the annotations

In [ ]:
#making the acanthomorph tree look good
import os, uuid
from ast import literal_eval
from ete2 import Tree, TreeStyle, AttrFace, NodeStyle, faces
import pandas as pd

def getAnnotations(tree, feature = "support"):
    #traverse tree and populate dictionary with id and feature as key, value
    featuredict = {}
    for node in tree.traverse():
        key = node.get_topology_id()
        if feature == "nodenumber":
            value = node.nodenumber
        else:
            value = node.support
        featuredict[key] = value
        
    return featuredict

def getCladeNodes(tree, subcladenode, show = False):
    #take tree and subclade node
    #return list of paml labeled subclades within parent subclade
    subtree = pamlLabelTree.search_nodes(support = subcladenode)[0]
    if show == True:
        subtree.show()
    subtreenodes = []
    for node in subtree.traverse():
        if node.pamllabel:
            subtreenodes.append(int(node.pamllabel))
    return subtreenodes    
    
workingdir = ("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/PAML_run_files/4_partitions/12_cali_4_parts_paml-output/combined-output/combined-annotated-tree/")
os.chdir(workingdir)

exaPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-75p-STDPART-1.5M-Burn25-FINAL.tre")
exaNoPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-75p-UNPART-1.5M-Burn25-FINAL.tre")
raxPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/RAxML.acanthomorph-no-chauliodius-75p-STDPART.tre")
raxNoPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/RAxML.acanthomorph-no-chauliodius-75p-UNPART.tre")
pamlLabelTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/PAML_run_files/4_partitions/12_cali_4_parts_paml-output/4-part-paml-labels.tree")



#root the raxml tree 
tt = raxPartTree
ancestor = "alepisaurus_ferox"
tt.set_outgroup(ancestor)
tt.get_tree_root().dist = 0.001

#get the exabayes_numbered tree. this is how I numbered the exabayes tree for figure 1
tt = exaPartTree
ancestor = "alepisaurus_ferox"
tt.set_outgroup(ancestor)
tt.get_tree_root().dist = 0.001
counter = 0
for node in tt.traverse("postorder"):
    node.add_features( nodenumber = str(counter) )# for calibrations
    counter += 1
    node.add_face( faces.TextFace(node.nodenumber, ftype="Arial", fsize=10, fstyle = "italic", fgcolor = "red"), column=0)


    
pamldict = {} #store numbers and paml names
for node in pamlLabelTree.traverse():
    node.add_features(bootstrap = None, posterior = None, mikelabel = None, pamllabel = None)
    #print node.get_topology_id()
    if node.is_leaf():
        num, genus, epi = node.name.split("_")
        species = "_".join([genus, epi])
        node.add_features(standard_name = species)
        pamldict[species] = node.name
        node.add_face( faces.TextFace(node.standard_name, ftype="Arial", fsize=14, fstyle = "italic"), column=0)
        node.name = species # make tip names match other tree files
        
        
posteriors = getAnnotations(exaPartTree)
bootstraps = getAnnotations(raxPartTree)
pamllabels = getAnnotations(pamlLabelTree)
mikelabels = getAnnotations(exaPartTree, "nodenumber")

for node in pamlLabelTree.traverse():  
    if not node.is_leaf():
        nid = node.get_topology_id()
        node.pamllabel = pamllabels[nid]
        if nid in posteriors.keys():
            node.posterior = posteriors[nid]
            node.add_face( faces.TextFace(node.posterior, ftype="Arial", fsize=10, fstyle = "italic", fgcolor = "red"), column=0)
            #faces.add_face_to_node(AttrFace("name"), node, column=0)
        if nid in bootstraps.keys():
            node.bootstrap = bootstraps[nid]
            node.add_face( faces.TextFace(node.bootstrap, ftype="Arial", fsize=10, fstyle = "italic", fgcolor = "blue"), column=0)           
        if nid in mikelabels.keys():
            node.mikelabel = mikelabels[nid]
            node.add_face( faces.TextFace(node.mikelabel, ftype="Arial", fsize=8, fgcolor = "green"), position = "branch-top",column=1)
            
    
        
###get duaghter node number for all subcaldes for figure 2
acanthomorph_subclades = {"paracanthopterygians" : 120, "percomorphs": 135,
                          "gobiomorphs": 139, "syngnathiforms": 155,
                          "pelagiaria": 160, "eupercaria":200,
                          "ovalentaria": 173, "caringimorphs": 183}

acanth_daughters = {} # hold subclade name and ALL subclade numbers within it
for clade in acanthomorph_subclades:
    daughters = getCladeNodes(pamlLabelTree, acanthomorph_subclades[clade])
    acanth_daughters[clade] = daughters
#writing to the working dir opened at start of script
acanth_daughters
ff = open("subclade_numbers.csv", "w")
for key in acanth_daughters.keys():
    for value in acanth_daughters[key]:
        #print "{}, {}".format(value, key)
        ff.write("{}, {}\n".format(value, key))
ff.close()
    
### put the fossil constraints on the tree, focus only on scheme 3 for labeling figure 1
infile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/12_cali_plecto_bounds.csv"
writedir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/Friedman_12_calibrations/plecto/"


scheme3 = {}

dd = pd.read_csv(infile, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)

#populate the scheme dictionaries
for cal_node in dd.index:
    min_age = dd.loc[cal_node][1] 
    max_age_3 = dd.loc[cal_node][4]
    scheme3[cal_node] = [min_age, max_age_3]

# from scheme 3 map mikelabels to pamllabels
mike_cali_nodes = scheme3.keys()
paml_cali_nodes = {} # search tree with mike_cali_nodes and populate this dict

for nn, bounds in scheme3.items():
    paml_node_number = pamlLabelTree.search_nodes(mikelabel = str(nn))[0].pamllabel
    paml_cali_nodes[paml_node_number] = bounds
    
ff = open("fig_1_constrained_nodes.csv", "w")
for key, value in paml_cali_nodes.items():
    lower = "{:.2f}".format(value[0])
    upper = "{:.2f}".format(value[1])
    print "{}, {}, {}\n".format(int(key), lower, upper)
    ff.write("{}, {}, {}\n".format(int(key), lower, upper))
ff.close()

##can stick these bounds on the pamllabelstree too
for node in pamlLabelTree.traverse():
    if not node.is_leaf():
        node.add_features(lower = None, upper = None)
        if node.pamllabel in paml_cali_nodes.keys():
            lower, upper = paml_cali_nodes[node.pamllabel][0], paml_cali_nodes[node.pamllabel][1]
            node.lower = "{:.2f} - ".format(lower)
            node.upper = "{:.2f}".format(upper)
            node.add_face( faces.TextFace(node.lower, ftype="Arial", fsize=8, fgcolor = "purple"), position = "branch-right",column=2)
            node.add_face( faces.TextFace(node.upper, ftype="Arial", fsize=8, fgcolor = "black"), position = "branch-right",column=3)

        
    

# in acanth_daughters.keys():
#     for value in acanth_daughters[key]:
#         #print "{}, {}".format(value, key)
#         ff.write("{}, {}\n".format(value, key))
# ff.close()


from ete2 import Tree, TreeStyle 
ts = TreeStyle() 

ts.show_leaf_name = False 
ts.show_branch_length = False 
ts.show_branch_support = True 
pamlLabelTree.show(tree_style=ts)
# #pamlLabelTree.get_leaf_names()
# # fourTrees = {
# # "exa_75_part" : exaPartTree, 
# # "exa_75_no_part" : exaNoPartTree, 
# # "rax_75_part" : raxPartTree,  
# # "rax_75_no_part" : raxNoPartTree 
# # }

193, 55.20, 102.08

163, 54.17, 91.55

196, 49.00, 65.17

133, 49.00, 101.62

230, 49.00, 73.67

174, 49.00, 88.47

221, 29.62, 85.97

211, 11.90, 50.18

120, 98.00, 128.82

124, 93.90, 128.04

125, 63.10, 93.51

190, 41.20, 55.86



AttributeError: '_TreeScene' object has no attribute 'selector'

AttributeError: '_TreeScene' object has no attribute 'selector'

In [78]:
# type(mike_cali_nodes[0])
# temp = []
# for node in pamlLabelTree.traverse():
#     if not node.is_leaf():
#         print type(node.mikelabel)
# #         temp.append(node.mikelabel)
# # for nn in mike_cali_nodes:
# #     print str(nn) in temp
# #     print type(nn)
# #     print type(temp[0])

In [91]:
tt.features

'dist'

In [84]:
ts = TreeStyle() 
ts.show_leaf_name = True 
ts.show_branch_length = False 
ts.show_branch_support = True 
raxPartTree.ladderize()
raxNoPartTree.show(tree_style=ts)

In [35]:
##need to root the exabayes trees, set root branch to 0, assign uuid
for name in fourTrees.keys():
    tt = fourTrees[name]
    ancestor = "alepisaurus_ferox"
    tt.set_outgroup(ancestor)
    tt.get_tree_root().dist = 0.001
    counter = 0
    for node in tt.traverse("postorder"):
        node.add_features( nodeid = str(uuid.uuid1() ) )
        node.add_features( nodenumber = str(counter) )# for calibrations
        node.add_features( order = None)
        counter += 1
#         if node.is_leaf():
#             node.name = node.name.capitalize().replace("_", " ")
#             #print node.name
#         if node.name == "Takifugu occelatus":
#             node.name = "Takifugu ocellatus"
#         if node.name == "Ostorhinchus nigrofasciatus":
#             node.name = "Ostorhinchus nigrofasciatus"
#         if node.name == "Sargocentron coruscum2":
#             node.name = "Sargocentron coruscum"
#             print "changed {}".format(node.name)


outfile = "4_27_2015_acanthomorh_renamed.tre"

tt = fourTrees["exa_75_part"]
ml = fourTrees["rax_75_part"] #just grabbing the ml tree for the bs values

# os.chdir(home)
# tt.write( features = ["nodeid", "nodenumber"], outfile = outfile)


rr = open("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas_tex/ETE_work/converted_pngs/ranks.txt").read()
fam_key = literal_eval(rr)
ranks = fam_key

attachSupport(tt, ml)

NameError: name 'fourTrees' is not defined

In [80]:
tt.features

'dist'